<b><font size=5><center>Исследование надежности заёмщиков</center></font></b>

### Суть проекта

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Описание данных

- `children` — количество детей в семье
- `days_employed` — общий трудовой стаж в днях
- `dob_years` — возраст клиента в годах
- `education` — уровень образования клиента
- `education_id` — идентификатор уровня образования
- `family_status` — семейное положение
- `family_status_id` — идентификатор семейного положения
- `gender` — пол клиента
- `income_type` — тип занятости
- `debt` — имел ли задолженность по возврату кредитов
- `total_income` — ежемесячный доход
- `purpose` — цель получения кредита

### Обзор данных

In [1]:
#Чтение и первичный обзор данных. 
#При поверхностном знакомстве уже видны некоторые недочеты в данных - разный регистр, слишком большое количество знаков после запятой и т.д.
import pandas as pd
data = pd.read_csv(r"C:\Users\mi\Downloads\data_borrowers.csv")
data.info()
data.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Заполнение пропусков

In [2]:
#При проверке пропусков видно, что в столбце с информацией о количестве дней трудоустройсва и общего дохода находятся одинаковое количество пропусков.
#Я посчитала долю пропусков к общему числу данных в столбцах. 
#Учитывая, что количество пропусков одинаковое в обоих столбцах, можно предположить, что оно возникло не случайно.
#В данном случае, т.к. это количественные данные, допустимо и разумно заменить пропуски на среднее значение, чтобы не искажать общую картину.

display(data.isna().sum())
print('Доля пропусков в столбце с длительностью трудоустройства {:.0%}'.format(data['days_employed'].isna().sum() / len(data['days_employed'])))
print('Доля пропусков в столбце с общим доходом {:.0%}'.format(data['total_income'].isna().sum() / len(data['total_income'])))  
data.loc[data['days_employed'].isna(),'days_employed'] = data['days_employed'].median()
data.loc[data['total_income'].isna(),'total_income'] = data['total_income'].median()

#Проверка:
display(data.isna().sum())


children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Доля пропусков в столбце с длительностью трудоустройства 10%
Доля пропусков в столбце с общим доходом 10%


children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Проверка данных на аномалии и исправления.

In [3]:
#Вручную обработаем по очереди наиболее важные колонки и колонки с слишком аномальными данными.
#Начнем с колонки дети. Здесь наблюдается присутствие аномальных данных = -1 и 20. Т.к. аномалий только 2, можно предположить, что это систематическая опечатка в виду близкого расположения клавиш. 
#Поэтому просто заменим их на наболее логичные данные = -1 на 1, а 20 на 2
display(data['children'].value_counts())
data.loc[data['children'] == -1, 'children'] = 1
data.loc[data['children'] == 20, 'children'] = 2
display(data['children'].value_counts())

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

In [4]:
#В количестве дней трудойстройства встречается очень много отрицательных и сишком больших данных.
#Предпоожим, что эта аномалия возникла в ходе слияния двух баз данных, и в одной из них данная колонка хранила информацию в часах. Сделаем данные пололжительными и приведем все к дням. 

data.loc[data['days_employed'] < 0, 'days_employed'] = abs((data.loc[data['days_employed'] < 0, 'days_employed']))
data['days_employed'] = data['days_employed'] / 24
display(data['days_employed'].head(10))

0      351.569709
1      167.700156
2      234.309275
3      171.864467
4    14177.753002
5       38.591076
6      119.966752
7        6.365815
8      288.744387
9       91.198185
Name: days_employed, dtype: float64

In [5]:
#если посмотреть на отдельные значения колонки возраста, то можно увидеть значение 0. Заменим это значение на среднее по столбцу, чтобы не искажать общую картину.
display(data['dob_years'].value_counts())
data.loc[data['dob_years'] == 0,'dob_years'] = int(data['dob_years'].median())
display(data['dob_years'].value_counts())

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
22    183
66    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

42    698
35    617
40    609
41    607
34    603
38    598
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
22    183
66    183
67    167
21    111
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

In [6]:
#В столбце с семейным положением аномалий нет. Просто приведем его значения к нижнему регистру для красоты:)
data['family_status'] = data['family_status'].str.lower()
data['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [7]:
display(data['gender'].value_counts())

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

### Изменение типов данных.

In [8]:
#Меняем вещественный тип данных в столбце с доходом на целочисленный.
data['total_income'] = data['total_income'].astype('int')
display(data['total_income'].head())

0    253875
1    112080
2    145885
3    267628
4    158616
Name: total_income, dtype: int32

### Удаление дубликатов.

In [9]:
#Приведение к нижнему регистру столбца с образованием. Проверка на наличие дубликатов.
#Т.к. дубликатов всего 72 из 21 тысячи, мы можем безболезненно их удалить из датафрейма без вреда для общей картины.
#Можно предположить что некоторые дубликаты в столбцах с длительностью трудоустройства и доходом образовались на месте пропусков из-за заполнения медианным значением.

data['education'] = data['education'].str.lower()
print('Дубликатов до:', data.duplicated().sum())
data = data.drop_duplicates().reset_index(drop=True)
print('Дубликатов после:', data.duplicated().sum())

Дубликатов до: 72
Дубликатов после: 0


### Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [10]:
#Формируем отдельные датафреймы со столбцами образования и семейного положения и их идентификаторами. 
#В исходном датафрейме удаляем столбцы образования и семейного положения. Проверяем выводом на экран.

ed_df = data[['education_id', 'education']]
ed_df = ed_df.drop_duplicates().reset_index(drop=True)
fam_df = data[['family_status_id', 'family_status']]
fam_df = fam_df.drop_duplicates().reset_index(drop=True)
data = data.drop(columns = ['family_status', 'education'], axis = 1) 
display(data.head())
display(ed_df.head())
display(fam_df.head())

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,351.569709,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,167.700156,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,234.309275,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,171.864467,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,14177.753002,53,1,1,F,пенсионер,0,158616,сыграть свадьбу


,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,не женат / не замужем


### Шаг 2.6. Категоризация дохода.

In [11]:
#Пишем функцию, которая присваивает категорию заемщикам на основании данных столбца дохода. 
#Применяем эту функцию и создаем новый сотлбец с категориями по доходу. Проверим выводом на экран.
def total_income_category(income):
    if income <= 30000:
        return 'E'
    if 30001 <= income <= 50000:
        return 'D'
    if 50001 <= income <= 200000:
        return 'C'
    if 200001 <= income <= 1000000:
        return 'B'
    return 'A'
data['total_income_category'] = data['total_income'].apply(total_income_category)
display(data.head(10))

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,351.569709,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,167.700156,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,234.309275,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,171.864467,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,14177.753002,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C
5,0,38.591076,27,0,1,M,компаньон,0,255763,покупка жилья,B
6,0,119.966752,43,0,0,F,компаньон,0,240525,операции с жильем,B
7,0,6.365815,50,1,0,M,сотрудник,0,135823,образование,C
8,2,288.744387,35,0,1,F,сотрудник,0,95856,на проведение свадьбы,C
9,0,91.198185,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи,C


### Категоризация целей кредита.

In [12]:
#Для удобства выведу все возможные значения столбца цели
display(data['purpose'].value_counts())

свадьба                                   791
на проведение свадьбы                     767
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

In [13]:
#Пишем функцию, которая присваивает категорию заемщикам на основании данных столбца целей кредита. 
#Применяем эту функцию и создаем новый сотлбец с категориями по целям. Проверим выводом на экран.

def purpose_category(purpose):
    if 'автомобил' in purpose:
        return 'операции с автомобилем'
    if 'недвижимост' in purpose or 'жиль' in purpose:
        return 'операции с недвижимостью'
    if 'свадьб' in purpose:
        return 'проведение свадьбы'
    if 'образовани' in purpose:
        return 'получение образования'
    
data['purpose_category'] = data['purpose'].apply(purpose_category)

display(data.head(10))

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,351.569709,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,167.700156,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,234.309275,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,171.864467,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,14177.753002,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
5,0,38.591076,27,0,1,M,компаньон,0,255763,покупка жилья,B,операции с недвижимостью
6,0,119.966752,43,0,0,F,компаньон,0,240525,операции с жильем,B,операции с недвижимостью
7,0,6.365815,50,1,0,M,сотрудник,0,135823,образование,C,получение образования
8,2,288.744387,35,0,1,F,сотрудник,0,95856,на проведение свадьбы,C,проведение свадьбы
9,0,91.198185,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи,C,операции с недвижимостью


### Ответы на вопросы

##### Вопрос 1:

Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [14]:
report_children = data.pivot_table(index='children', columns='debt', values='total_income', aggfunc='count').fillna(0) #т.к. считаем строки, то не имеет значение, по какому сотолбцу. Поэтому считаем по total_income
report_children.columns = ['Нет задолженности', 'Задолженность']
report_children['% задолженности'] = report_children['Задолженность'] / report_children['Нет задолженности']
report_children['% задолженности'] = report_children['% задолженности'].fillna(0)
display(report_children.sort_values(by='% задолженности').style.format({'% задолженности': '{:.2%}', 'Задолженность': '{:.0f}', 'Нет задолженности': '{:.0f}'}))

,Нет задолженности,Задолженность,% задолженности
children,,,
5,9,0,0.00%
0,13027,1063,8.16%
3,303,27,8.91%
1,4410,445,10.09%
2,1926,202,10.49%
4,37,4,10.81%


##### Вывод 1:

Можно сказать, что зависимость есть, но несущественная. Разброс процентов задолженности среди категорий совсем небольшой.
Но смотря на имеющуюся картину, можно сказать, что у бездетных шанс не выплатить кредит меньше, чем у других заемщиков. Также можно заметить полное отсутствие задолженности у заемщиков с пятью детьми, но и выборка там меньше:)

##### Вопрос 2:

Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [15]:
report_fs = data.pivot_table(index='family_status_id', columns='debt', values='total_income', aggfunc='count').fillna(0)
report_fs.columns = ['Нет задолженности', 'Задолженность']
report_fs['% задолженности'] = report_fs['Задолженность'] / report_fs['Нет задолженности']
report_fs['% задолженности'] = report_fs['% задолженности'].fillna(0)
report_fs_merged = fam_df.merge(report_fs, on = 'family_status_id')
display(report_fs_merged.sort_values(by='% задолженности').style.format({'% задолженности': '{:.2%}', 'Задолженность': '{:.0f}', 'Нет задолженности': '{:.0f}'}))

,family_status_id,family_status,Нет задолженности,Задолженность,% задолженности
2,2,вдовец / вдова,896,63,7.03%
3,3,в разводе,1110,85,7.66%
0,0,женат / замужем,11408,931,8.16%
1,1,гражданский брак,3762,388,10.31%
4,4,не женат / не замужем,2536,274,10.80%


##### Вывод 2:

Здесь прослеживается четкая зависимость - у людей не в браке или в гражданском браке шанс не выплатить кредит возрастает.
Самые надежные заемщики в данной выборке - вдовцы/вдовы или люди в разводе.

##### Вопрос 3:

Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [16]:
report_income = data.pivot_table(index='total_income_category', columns='debt', values='total_income', aggfunc='count').fillna(0)
report_income.columns = ['Нет задолженности', 'Задолженность']
report_income['% задолженности'] = report_income['Задолженность'] / report_income['Нет задолженности']
report_income['% задолженности'] = report_income['% задолженности'].fillna(0)
display(report_income.sort_values(by='% задолженности').style.format({'% задолженности': '{:.2%}', 'Задолженность': '{:.0f}', 'Нет задолженности': '{:.0f}'}))

,Нет задолженности,Задолженность,% задолженности
total_income_category,,,
D,329,21,6.38%
B,4685,356,7.60%
A,23,2,8.70%
C,14655,1360,9.28%
E,20,2,10.00%


##### Вывод 3:

Однозначно можно сказать, что наименьшей платежеспособностью отличаются люди с наименьшим доходом (что вполне логично).
Также логично было бы, если бы самый маленький процент задолженности был у людей с наибольшим доходом - но это не так.
У данной категории заемщиков средняя платежеспособность.
Самыми надежными оказались заемщики с доходом выше среднего (от 200 тыс до миллиона).
В целом, сложно проследить какие-либо паттерны, поэтому скажу, что прямой зависимости возврата кредита от дохода нет.

##### Вопрос 4:

Как разные цели кредита влияют на его возврат в срок?

In [17]:
report_purpose = data.pivot_table(index='purpose_category', columns='debt', values='total_income', aggfunc='count').fillna(0)
report_purpose.columns = ['Нет задолженности', 'Задолженность']
report_purpose['% задолженности'] = report_purpose['Задолженность'] / report_purpose['Нет задолженности']
report_purpose['% задолженности'] = report_purpose['% задолженности'].fillna(0)
display(report_purpose.sort_values(by='% задолженности').style.format({'% задолженности': '{:.2%}', 'Задолженность': '{:.0f}', 'Нет задолженности': '{:.0f}'}))

,Нет задолженности,Задолженность,% задолженности
purpose_category,,,
операции с недвижимостью,10029,782,7.80%
проведение свадьбы,2137,186,8.70%
получение образования,3643,370,10.16%
операции с автомобилем,3903,403,10.33%


##### Вывод 4:

Операции с недвижимостью - самая безопасная цель кредита. Заемщиков с данной целью в разы больше других, и они отличаются самым низких процентом задолженности.
Самым высоким процентом задолженности отличаются операции с автомобилем.

## Общий вывод:

В целом, можно сказать, что самые надежные заемщики - это люди, уже побывавшие в браке или находящиеся в нем, без детей, берущие займ для операций с недвижимостью и имеющие уровень дохода от 200 тыс до миллиона. 
Мне кажется, стоило бы еще включить в анализ сумму займов, чтобы найти зависимость между суммой займа, доходом и наличием просрочек. Такая картина была бы более точной. 